In [ ]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 5.5MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 5.7 MB/s 
     |████████████████████████████████| 636 kB 16.2 MB/s 
     |████████████████████████████████| 1.5 MB 13.7 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 56 kB 3.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 5.2 MB/s 
     |████████████████████████████████| 1.0 MB 35.8 MB/s 
     |████████████████████████████████| 61 kB 6.4 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=ef132508c6ea8c93b4b032dad64aa7ba593950d35bfbf00e174c48873cc45452
  Stored in directory: /root/.cac

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp

In [ ]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_ca:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [ ]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess:v.0.1',
        arguments = ['--data', data],
        file_outputs={'clean_data': '/preprocessing/clean_data'     
        }
    )

In [ ]:
def train_op(clean_data):
    return dsl.ContainerOp(
        name = 'Train data',
        image = 'mavencodevv/train:v.0.1',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'fit_model': '/train_model/fit_model' 
        }
    )

In [ ]:
def test_op(clean_data, fit_model):
    return dsl.ContainerOp(
        name = 'Test model',
        image = 'mavencodevv/test_ca:v.0.1',
        arguments = ['--clean_data', clean_data,
                    '--fit_model', fit_model
            ],
        file_outputs={
            'metrics': '/test_model/metrics' 
        }
    )

In [ ]:
def results(metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/metrics_ld:v.0.1',
        arguments = ['--metrics', metrics
            ] 
        
    )

In [ ]:
@dsl.pipeline(
    name='Churn classiifer pipeline',
   description='An ML reusable pipeline that performs customer segmentation to determine customers with high risk of leaving a telcommunication company .'
)
# Define parameters to be fed into pipeline
def churn_classifier_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
    _train_op = train_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    
    _test_op = test_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data']),
        dsl.InputArgumentPath(_train_op.outputs['fit_model'])).after(_train_op)
    _results = results(
        dsl.InputArgumentPath(_test_op.outputs['metrics'])).after(_test_op)
    

In [ ]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'churn_classifier_pipeline'
#kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline,  
#  '{}.zip'.format(experiment_name))
kfp.compiler.Compiler().compile(churn_classifier_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [ ]:
client = kfp.Client()
client.create_run_from_pipeline_func(churn_classifier_pipeline, arguments={})

RunPipelineResult(run_id=c0665dc5-a861-4430-b876-ae2ae1c0a4aa)